In [1]:
import os

In [2]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\client-subscription-prediction'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluation:
    root_dir : Path
    test_data_path: Path
    model_path: Path
    all_params : dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    






In [ ]:
from clientClassifier.constants import *    
from clientClassifier.utils.common import read_yaml, create_directories 

SyntaxError: invalid syntax (233654067.py, line 2)